# **Imported Library**

In [1]:
import pandas as pd
import nltk as nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') # Download wordnet, which is required by WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns',None)
pd.set_option('display.width',1000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Imported Datasets**

In [3]:
datapath_tweets_labeled=('/content/drive/My Drive/dataset/twitterSentimentAnalystDataset/tweets_labeled.csv')
datapath_data=('/content/drive/My Drive/dataset/twitterSentimentAnalystDataset/tweets_21.csv')
tweets_labeled=pd.read_csv(datapath_tweets_labeled,sep=",")
tweets_not_labeled=pd.read_csv(datapath_data,sep=",")



# **Feature Engineering**

In [4]:
tweets_labeled.head(50)

,tweet_id,tweet,date,label
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-24 21:41:37+00:00,1
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-15 21:00:18+00:00,1
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-13 21:21:36+00:00,1
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 15:03:47+00:00,1
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 20:22:32+00:00,1
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 18:28:24+00:00,1
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 01:58:49+00:00,1
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 19:51:21+00:00,1
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 05:49:22+00:00,1
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 19:12:12+00:00,1


In [5]:
tweets_not_labeled.head(10)

,tweet_id,tweet,date
0,1344798002490314752,Kardesim (12) sevdigi cocuga 00:00 da ilan-i a...,2021-01-01 03:10:03
1,1344799527673470977,Bu yılbaşı da saat tam 00:00'da swni seviyorum...,2021-01-01 03:16:07
2,1344799907719348226,tam 00.00da naptınız ben her yıl tam bu vakit ...,2021-01-01 03:17:37
3,1344800782802165762,00:00'da havai fişek gösterisi yapıldı tam dib...,2021-01-01 03:21:06
4,1344805589990453249,31 aralık saat 00.00 da yeni yılımızı kutlayan...,2021-01-01 03:40:12
5,1344838906609606658,Saat 23:00 olmadan sızmışım. Bütün senenin yo...,2021-01-01 05:52:35
6,1344845204965232640,yalnızlıktan ve sıkıntıdan şimdide kitaplara s...,2021-01-01 06:17:37
7,1344856870759129088,askerde 9da yat emri alıp uyuyup gece tam 00.0...,2021-01-01 07:03:58
8,1344893883554164736,"Dertlerimi yazıyorum buraya, herkes beğeniyor,...",2021-01-01 09:31:03
9,1344898883265114112,minho tam 00.00'da bize mutlu yıllar demiş ve ...,2021-01-01 09:50:55


In [6]:
# Deleting rows with no data
tweets_labeled.dropna(axis=0, inplace=True)




# If the 'date' column already contains timezone information, use tz_convert instead of tz_localize
tweets_labeled['date'] = pd.to_datetime(tweets_labeled['date'])


# Convert to desired timezone
tweets_labeled['date'] = tweets_labeled['date'].dt.tz_convert('Europe/Istanbul')


tweets_labeled.head(10)

,tweet_id,tweet,date,label
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37+03:00,1
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18+03:00,1
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36+03:00,1
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47+03:00,1
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32+03:00,1
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 21:28:24+03:00,1
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 04:58:49+03:00,1
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 22:51:21+03:00,1
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 08:49:22+03:00,1
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 22:12:12+03:00,1


In [7]:
# Adding Month column and splitting months in date
tweets_labeled['month']=tweets_labeled['date'].dt.month_name()
tweets_labeled['month']=tweets_labeled['month'].replace({'December':'Aralık',
                                                         'January':'Ocak',
                                                         'February':'Şubat',
                                                         'March':'Mart',
                                                         'April':'Nisan',
                                                         'May':'Mayıs',
                                                         'June':'Haziran',
                                                         'July':'Temmuz',
                                                         'August':'Ağustos',
                                                         'September':'Eylül',
                                                         'October':'Ekim',
                                                         'November':'Kasım',
                                                         })

tweets_labeled.head(10)



,tweet_id,tweet,date,label,month
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37+03:00,1,Aralık
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18+03:00,1,Haziran
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36+03:00,1,Haziran
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47+03:00,1,Şubat
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32+03:00,1,Şubat
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 21:28:24+03:00,1,Şubat
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 04:58:49+03:00,1,Şubat
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 22:51:21+03:00,1,Aralık
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 08:49:22+03:00,1,Ekim
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 22:12:12+03:00,1,Haziran


In [8]:
# Add a season column and transfer data
seasons={'Ocak':'Kış',
        'Şubat':'Kış',
        'Mart':'İlkbahar',
        'Nisan':'İlkbahar',
        'Mayıs':'İlkbahar',
        'Haziran':'Yaz',
        'Temmuz':'Yaz',
        'Ağustos':'Yaz',
        'Eylül':'Sonbahar',
        'Ekim':'Sonbahar',
        'Kasım':'Sonbahar',
        'Aralık':'Kış'
          }
tweets_labeled["season"]=tweets_labeled["month"].map(seasons)
tweets_labeled.head(10)

,tweet_id,tweet,date,label,month,season
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37+03:00,1,Aralık,Kış
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18+03:00,1,Haziran,Yaz
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36+03:00,1,Haziran,Yaz
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47+03:00,1,Şubat,Kış
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32+03:00,1,Şubat,Kış
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 21:28:24+03:00,1,Şubat,Kış
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 04:58:49+03:00,1,Şubat,Kış
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 22:51:21+03:00,1,Aralık,Kış
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 08:49:22+03:00,1,Ekim,Sonbahar
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 22:12:12+03:00,1,Haziran,Yaz


In [9]:
# Creating the day variable

tweets_labeled["days"]=[date.strftime('%A') for date in tweets_labeled["date"]]
tweets_labeled["days"] = tweets_labeled["days"].replace({'Monday':'Pazartesi',
                                                         'Tuesday':'Salı',
                                                         'Wednesday':'Çarşamba',
                                                         'Thursday':'Perşembe',
                                                         'Friday':'Cuma',
                                                         'Saturday':'Cumartesi',
                                                         'Sunday':'Pazar'})

tweets_labeled.head(10)



,tweet_id,tweet,date,label,month,season,days
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37+03:00,1,Aralık,Kış,Pazar
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18+03:00,1,Haziran,Yaz,Perşembe
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36+03:00,1,Haziran,Yaz,Salı
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47+03:00,1,Şubat,Kış,Pazar
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32+03:00,1,Şubat,Kış,Pazar
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 21:28:24+03:00,1,Şubat,Kış,Pazar
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 04:58:49+03:00,1,Şubat,Kış,Cumartesi
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 22:51:21+03:00,1,Aralık,Kış,Perşembe
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 08:49:22+03:00,1,Ekim,Sonbahar,Pazartesi
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 22:12:12+03:00,1,Haziran,Yaz,Cuma


In [10]:
# Dividing the day into six parts with 4 hour intervals
tweets_labeled["hour"] = tweets_labeled["date"].dt.hour
tweets_labeled["4hour_interval"]=(tweets_labeled["hour"]//2)*2
interval = {0: '0-2',
            2: '2-4',
            4: '4-6',
            6: '6-8',
            8: '8-10',
            10: '10-12',
            12: '12-14',
            14: '14-16',
            16: '16-18',
            18: '18-20',
            20: '20-22',
            22: '22-24'
            }

tweets_labeled['4hour_interval'] = tweets_labeled['4hour_interval'].map(interval)
tweets_labeled["time_interval"] = tweets_labeled["4hour_interval"].replace({"0-2": "22-02",
                                                   "22-24": "22-02",
                                                   "2-4": "02-06",
                                                   "4-6": "02-06",
                                                   "6-8": "06-10",
                                                   "8-10": "06-10",
                                                   "10-12": "10-14",
                                                   "12-14": "10-14",
                                                   "14-16": "14-18",
                                                   "16-18": "14-18",
                                                   "18-20": "18-22",
                                                   "20-22": "18-22"})

tweets_labeled.drop(["4hour_interval", "hour"], axis=1, inplace=True)

tweets_labeled.head(50)

,tweet_id,tweet,date,label,month,season,days,time_interval
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37+03:00,1,Aralık,Kış,Pazar,22-02
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18+03:00,1,Haziran,Yaz,Perşembe,22-02
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36+03:00,1,Haziran,Yaz,Salı,22-02
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47+03:00,1,Şubat,Kış,Pazar,18-22
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32+03:00,1,Şubat,Kış,Pazar,22-02
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 21:28:24+03:00,1,Şubat,Kış,Pazar,18-22
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 04:58:49+03:00,1,Şubat,Kış,Cumartesi,02-06
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 22:51:21+03:00,1,Aralık,Kış,Perşembe,22-02
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 08:49:22+03:00,1,Ekim,Sonbahar,Pazartesi,06-10
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 22:12:12+03:00,1,Haziran,Yaz,Cuma,22-02


In [11]:
# Separating the data in the label column into "negative, positive and neutral"
tweets_labeled["label"].replace(1, value="pozitif", inplace=True)
tweets_labeled["label"].replace(-1, value="negatif", inplace=True)
tweets_labeled["label"].replace(0, value="nötr", inplace=True)

tweets_labeled.head(10)


<ipython-input-11-d90ffda60276>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tweets_labeled["label"].replace(1, value="pozitif", inplace=True)


,tweet_id,tweet,date,label,month,season,days,time_interval
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37+03:00,pozitif,Aralık,Kış,Pazar,22-02
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18+03:00,pozitif,Haziran,Yaz,Perşembe,22-02
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36+03:00,pozitif,Haziran,Yaz,Salı,22-02
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47+03:00,pozitif,Şubat,Kış,Pazar,18-22
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32+03:00,pozitif,Şubat,Kış,Pazar,22-02
5,1490391945142751233,Halbuki sevmek biri için ölmek değil onun için...,2022-02-06 21:28:24+03:00,pozitif,Şubat,Kış,Pazar,18-22
6,1489780520359759873,Gülmek için bir tesadüf\nBeklemenin anlamı yok...,2022-02-05 04:58:49+03:00,pozitif,Şubat,Kış,Cumartesi,02-06
7,1608551263037435904,"✨2023’te sağlıklı,güzel kararlar,mutluluk,sept...",2022-12-29 22:51:21+03:00,pozitif,Aralık,Kış,Perşembe,22-02
8,1584421772166037505,Depresyon ve anksiyeteyle uğraşırken arkadaşla...,2022-10-24 08:49:22+03:00,pozitif,Ekim,Sonbahar,Pazartesi,06-10
9,1532802348350840836,Bisikletimdeki ledler ve ışıklı kulaklığım ile...,2022-06-03 22:12:12+03:00,pozitif,Haziran,Yaz,Cuma,22-02


# Veriyi Hazırlama ve Logistic Regression

In [12]:
## DATA PREPARATION tweets_labeled.csv
def data_preparation(data):


  """
    Data is prepared for the model,
    Data is converted to lower case letters
    Numerical values ​​are extracted
    Punctuation marks are extracted
    \n marks are extracted
    Stop Words are extracted
    Lemmatization is done

    Parameters
    ----------
        data : dataframe



    """


  # Convert to lower case
  tweets_labeled["tweet"]=tweets_labeled["tweet"].str.lower()

  # Subtracting numerical values
  tweets_labeled["tweet"]=tweets_labeled["tweet"].str.replace('\d',' ',regex=True)

  # punctuation removal
  tweets_labeled["tweet"]=tweets_labeled["tweet"].str.replace('[^\w\s]','',regex=True)

  # Removing \n signs
  tweets_labeled["tweet"] = tweets_labeled["tweet"].str.replace('\n',' ',regex=True)

  # Stop Words removal of words
  sw = stopwords.words('turkish')
  tweets_labeled["tweet"]=tweets_labeled["tweet"].apply(lambda x : " ".join(x for x in str(x).split()if x not in sw))

  # Lemmatization
  tweets_labeled["tweet"]=tweets_labeled["tweet"].apply(lambda x: " ".join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))




data_preparation(tweets_labeled)
tweets_labeled["tweet"]

# Label Encoder
tweets_labeled["label"]=LabelEncoder().fit_transform(tweets_labeled["label"])
y=tweets_labeled["label"]


# TF-IDF Model
tf_idf_word_vectorizer= TfidfVectorizer()
x=tf_idf_word_vectorizer.fit_transform(tweets_labeled["tweet"])
log_model = LogisticRegression(max_iter=10000).fit(x, y)
print(cross_val_score(log_model,
                x,
                y,
                scoring="accuracy",
                cv=10).mean())



0.6657925187091853


# Tahmin verilerini hazırlama ve tahmin etme

In [13]:
## DATA PREPARATION tweets_21.csv

def text_preprocessing(data):


  """
    Data is prepared for the model,
    Data is converted to lower case letters
    Numerical values ​​are extracted
    Punctuation marks are extracted
    \n marks are extracted
    Stop Words are extracted
    Lemmatization is done

    Parameters
    ----------
        data : dataframe



    """

  # Convert to lower case
  data["tweet"]=data["tweet"].str.lower()

  # Subtracting numerical values
  data["tweet"]=data["tweet"].str.replace('\d',' ',regex=True)


  # punctuation removal
  data["tweet"]=data["tweet"].str.replace('[^\w\s]','',regex=True)


  # Removing \n signs
  data["tweet"] = data["tweet"].str.replace('\n',' ',regex=True)


  # Stop Words removal of words

  sw = stopwords.words('turkish')
  data["tweet"]=data["tweet"].apply(lambda x : " ".join(x for x in str(x).split()if x not in sw))

  # Rare Words
  temp_df = pd.Series(''.join(data["tweet"]).split()).value_counts()
  drops = temp_df[temp_df <=3]
  data["tweet"]=data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in drops))

  # Lemmatization
  data["tweet"]=data["tweet"].apply(lambda x: " ".join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))


  data.dropna(axis=0, inplace=True)





text_preprocessing(tweets_not_labeled)
tweets_not_labeled["tweet"]


,tweet
0,kardesim sevdigi cocuga ask cocuk simdi salak ...
1,yılbaşı saat tam seviyorum mesajı gelmedi
2,tam ben yıl tam vakit dinlediğim şarkıyı açtım...
3,havai fişek yapıldı tam ancak kafamı bile öyle...
4,aralık saat yeni kutlayan birisi yine olmadı f...
...,...
13267,yarın gece saat kimse yeni yıl mesajı seviniri...
13268,x yılbaşı akşamı sevdiğim kişiyle yeni yıla ar...
13269,yarın tam kime mesaj atıp mutlu edeyim şimdide...
13270,lütfen gece a huzurlu girmek istiyorum yeni yı...


In [14]:
# Guess and transfer the result to the new label
tf_idf_word_vectorizer = TfidfVectorizer()
x_train = tf_idf_word_vectorizer.fit_transform(tweets_labeled["tweet"]) # Fit on labeled data
x_test = tf_idf_word_vectorizer.transform(tweets_not_labeled["tweet"])
predictions = log_model.predict(x_test)
tweets_not_labeled["label"] = predictions





In [15]:
tweets_not_labeled["label"].value_counts()



,count
label,
1,11188
0,1092
2,992


In [16]:
# Example tweet
import re
new_tweet = "Hayat çok güzel"
new_tweet = new_tweet.lower()
new_tweet = re.sub(r'[^\w\s]', '', new_tweet)
new_tweet = re.sub(r'\d', '', new_tweet)
new_tweet = pd.DataFrame({'tweet': [new_tweet]})

text_preprocessing(new_tweet)
new_tweet_transformed = tf_idf_word_vectorizer.transform(new_tweet["tweet"])
prediction = log_model.predict(new_tweet_transformed)
prediction

array([1])